<a href="https://colab.research.google.com/github/prasi09/AI_Reports/blob/main/7_A_star_8_puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import heapq

class PuzzleSolver:
    def __init__(self, start, goal):  # <-- Added self here
        self.start = start
        self.goal = goal
        self.rows = len(start)
        self.cols = len(start[0])

    def heuristic(self, state):
        #Calculate the Manhattan distance heuristic.
        distance = 0
        goal_flat = [val for row in self.goal for val in row]  # Flatten for lookup
        for i in range(self.rows):
            for j in range(self.cols):
                value = state[i][j]
                if value != 0:  # Ignore the blank tile
                    goal_index = goal_flat.index(value)
                    goal_x, goal_y = divmod(goal_index, self.cols)
                    distance += abs(i - goal_x) + abs(j - goal_y)
        return distance

    def get_neighbors(self, state):
        #Generate neighboring states.
        neighbors = []
        x, y = next((i, j) for i, row in enumerate(state) for j, val in enumerate(row) if val == 0)
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.rows and 0 <= ny < self.cols:
                new_state = [row[:] for row in state]
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
                neighbors.append(new_state)
        return neighbors

    def solve(self):
        #Solve the puzzle using A*
        start_flat = tuple(val for row in self.start for val in row)
        goal_flat = tuple(val for row in self.goal for val in row)

        pq = [(0, self.start, 0, None)]  # Priority queue: (f, state, g, parent)
        visited = {start_flat}
        came_from = {}

        while pq:
            _, current, g, parent = heapq.heappop(pq)

            if current == self.goal:
                return self.reconstruct_path(came_from, current)

            for neighbor in self.get_neighbors(current):
                neighbor_flat = tuple(val for row in neighbor for val in row)
                if neighbor_flat not in visited:
                    visited.add(neighbor_flat)
                    came_from[neighbor_flat] = current
                    f = g + 1 + self.heuristic(neighbor)  # f = g + h
                    heapq.heappush(pq, (f, neighbor, g + 1, current))

        return None, []  # No solution found, return empty moves

    def reconstruct_path(self, came_from, current):
        #Reconstruct the path and name the moves.
        path = []
        moves = []  # Store the moves
        while current:
            path.append(current)
            parent = came_from.get(tuple(val for row in current for val in row))
            if parent:  # Check if parent exists (not the start state)
                move = self.get_move_name(parent, current)
                moves.append(move)  # Store the move name
            current = parent
        return path[::-1], moves[::-1]  # Reverse both

    def get_move_name(self, parent, child):
        #Determine the move name based on parent and child states.
        px, py = next((i, j) for i, row in enumerate(parent) for j, val in enumerate(row) if val == 0)
        cx, cy = next((i, j) for i, row in enumerate(child) for j, val in enumerate(row) if val == 0)

        if cx > px:
            return "Down"
        elif cx < px:
            return "Up"
        elif cy > py:
            return "Right"
        else:
            return "Left"

# Example usage
start_state = [[1, 0, 3], [4, 2, 5], [7, 8, 6]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

solver = PuzzleSolver(start_state, goal_state)
solution, moves = solver.solve()  # Get both path and moves

if solution:
    print("Start State:")
    for row in start_state:
        print(row)
    print()

    print("Goal State:")
    for row in goal_state:
        print(row)
    print()

    print("Solution found in", len(solution) - 1, "steps:")
    for i, step in enumerate(solution):
        for row in step:
            print(row)
        print("Move:", moves[i - 1] if i > 0 else "Start")  # Print move name
        print()
else:
    print("No solution found.")

Start State:
[1, 0, 3]
[4, 2, 5]
[7, 8, 6]

Goal State:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Solution found in 3 steps:
[1, 0, 3]
[4, 2, 5]
[7, 8, 6]
Move: Start

[1, 2, 3]
[4, 0, 5]
[7, 8, 6]
Move: Down

[1, 2, 3]
[4, 5, 0]
[7, 8, 6]
Move: Right

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
Move: Down

